In [56]:
import os
import numpy as np
import numpy.ma as ma
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm
import pandas as pd
import iris
import iris.quickplot as qplt
import iris.plot as iplt
import cartopy.crs as ccrs
import iris.analysis.calculus as calculus
from iris.coord_categorisation import add_year, add_month_number, add_day_of_month, add_day_of_year, add_season_number
import warnings
warnings.filterwarnings('ignore')

%matplotlib agg

In [57]:
def return_indices_of_a(a, b):
  b_set = set(b)
  return [int(i) for i, v in enumerate(a) if v in b_set]

In [58]:
def prepare_calendar(cube):
    # Setting up the dates on data
    if not cube.coords('year'):
        iris.coord_categorisation.add_year(cube, 'time', name='year')
    if not cube.coords('month_number'):
        iris.coord_categorisation.add_month_number(cube, 'time', name='month_number')
    if not cube.coords('day_of_month'):
        iris.coord_categorisation.add_day_of_month(cube, 'time', name='day_of_month')
    if not cube.coords('day_of_year'):
        iris.coord_categorisation.add_day_of_year(cube, 'time', name='day_of_year')
    return cube

In [67]:
def plot_precip_monthly_means(runs):
    precip_monthly_clim_data = []
    for run in runs:
        runid = run['runid']
        data_root = os.path.join(run['data_retrieve_dir'], runid)
        if runid=='obs':
            precip_cube = iris.load_cube(os.path.join(data_root, 'GPM_PRECIP_SEA_24h_mean_2000_2020.nc'))
            #precip_cube = iris.load_cube('/project/MJO_GCSS/SoutheastAsia_data/TRMM/daily/3B42_daily.1998_2012_Global.nc')
        else:
            precip_cube = iris.load_cube(os.path.join(data_root, runid + '_PRECIP.pp.nc'))
            precip_cube.convert_units('kg m-2 day-1')
        precip_cube = precip_cube.intersection(longitude=(90, 130), latitude=(-10,25))
        precip_cube = prepare_calendar(precip_cube)
        
        precip_daily_climate = precip_cube.aggregated_by(['month_number'],iris.analysis.MEAN)
        
        precip_monthly_clim_data.append(precip_daily_climate.intersection(longitude=(95, 115), 
                                                                          latitude=(-10,15)).collapsed(('latitude', 'longitude'), 
                                                                                                       iris.analysis.MEAN))
        
    
    fig = plt.figure(1, figsize=(12, 5), dpi=100)
    for i, run in enumerate(runs):
        plt.bar(precip_monthly_clim_data[i].coord('month_number').points+i*0.2, 
                precip_monthly_clim_data[i].data, alpha=0.6, width=0.2, label=run['label'])
    plt.title('95-115E, 10S-15N')
    plt.legend()
    plt.gca().set_xticks(np.arange(1,13)+0.3)
    plt.gca().set_xticklabels(['J','F','M','A', 'M', 'J', 'J', 'A', 'S', 'O', 'N', 'D'])
    #plt.show()
    plt.savefig('Precip_monthly_climate.png')
    plt.close()

In [61]:
def create_dates_df(cube):
    cube = prepare_calendar(cube)
    cube_dates_df = pd.concat([pd.DataFrame(cube.coord('year').points),
                          pd.DataFrame(cube.coord('month_number').points), 
                          pd.DataFrame(cube.coord('day_of_month').points)], axis=1)
    cube_dates_df.columns = ['year', 'month', 'day']
    return cube_dates_df

def precip_mean_var_season(precip_cube, season_months = [5,6,7,8, 9, 10], outfile_basename=None):
    precip_cube_dates_df = create_dates_df(precip_cube)
    season_df = precip_cube_dates_df.loc[(precip_cube_dates_df['month'].isin(season_months))]
    inds = [ind for ind in season_df.index]
    seasonal_mean = precip_cube[inds].collapsed('time', iris.analysis.MEAN)
    seasonal_variance = precip_cube[inds].collapsed('time', iris.analysis.VARIANCE)
    
    if not outfile_basename is None:
        outfile = outfile_basename+'_mean.nc'
        iris.save(seasonal_mean, outfile)
        print('Mean Written %s' %outfile)
        outfile = outfile_basename+'_variance.nc'
        iris.save(seasonal_variance, outfile)
        print('Variance Written %s' %outfile)
    return seasonal_mean, seasonal_variance

In [85]:
months_labels = ['J','F','M','A', 'M', 'J', 'J', 'A', 'S', 'O', 'N', 'D']
season_months = [6,7,8]
''.join([months_labels[i-1] for i in season_months])

'JJA'

In [62]:
def plot_precip_winter_summer_maps(runs, filt=False):
    for run in runs:
        runid = run['runid']
        print(run['label'])
        data_root = os.path.join(run['data_retrieve_dir'], runid)
        if runid=='obs':
            if filt:
                precip_cube = iris.load_cube(os.path.join(data_root, 'GPM_PRECIP_SEA_24h_mean_2000_2020_filt.nc'))
            else:
                precip_cube = iris.load_cube(os.path.join(data_root, 'GPM_PRECIP_SEA_24h_mean_2000_2020.nc'))
                #precip_cube = iris.load_cube('/project/MJO_GCSS/SoutheastAsia_data/TRMM/daily/3B42_daily.1998_2012_Global.nc')
        else:
            if filt:
                precip_cube = iris.load_cube(os.path.join(data_root, runid + '_PRECIP_filt.nc'))
            else:
                precip_cube = iris.load_cube(os.path.join(data_root, runid + '_PRECIP.pp.nc'))
                
            precip_cube.convert_units('kg m-2 day-1')
        precip_cube = precip_cube.intersection(longitude=(90, 130), latitude=(-10,25))
        precip_cube = prepare_calendar(precip_cube)
        
        if filt:
            #outfile_basename = os.path.join('/project/MJO_GCSS/hadgem3/data/SEAPy_output/%s/%s_NDJFM_precipitation_flux_filt' %(runid, runid))
            outfile_basename = os.path.join('/project/MJO_GCSS/hadgem3/data/SEAPy_output/%s/%s_DJF_precipitation_flux_filt' %(runid, runid))
        else:
            #outfile_basename = os.path.join('/project/MJO_GCSS/hadgem3/data/SEAPy_output/%s/%s_NDJFM_precipitation_flux' %(runid, runid))
            outfile_basename = os.path.join('/project/MJO_GCSS/hadgem3/data/SEAPy_output/%s/%s_DJF_precipitation_flux' %(runid, runid))
        #winter_mean, winter_var = precip_mean_var_season(precip_cube, season_months=[11,12,1,2,3], outfile_basename=outfile_basename)
        
        winter_mean, winter_var = precip_mean_var_season(precip_cube, season_months=[12,1,2], outfile_basename=outfile_basename)
        
        if filt:
            #outfile_basename = os.path.join('/project/MJO_GCSS/hadgem3/data/SEAPy_output/%s/%s_MJJAS_precipitation_flux_filt' %(runid, runid))
            outfile_basename = os.path.join('/project/MJO_GCSS/hadgem3/data/SEAPy_output/%s/%s_JJA_precipitation_flux_filt' %(runid, runid))
        else:
            #outfile_basename = os.path.join('/project/MJO_GCSS/hadgem3/data/SEAPy_output/%s/%s_MJJAS_precipitation_flux' %(runid, runid))
            outfile_basename = os.path.join('/project/MJO_GCSS/hadgem3/data/SEAPy_output/%s/%s_JJA_precipitation_flux' %(runid, runid))
        #summer_mean, summer_var = precip_mean_var_season(precip_cube, season_months=[5,6,7,8,9], outfile_basename=outfile_basename)
        summer_mean, summer_var = precip_mean_var_season(precip_cube, season_months=[6,7,8], outfile_basename=outfile_basename)

        fig = plt.figure(1, figsize=(12, 7), dpi=100)
        plt.subplot(111)
        ax = plt.axes(projection=ccrs.PlateCarree())
        levels = np.arange(-20, 20, 2)
        cmap = plt.get_cmap('RdBu')
        norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)
        cf = iplt.pcolormesh(winter_mean-summer_mean, cmap=cmap, norm=norm)
        plt.gca().coastlines()
        plt.colorbar(cf, orientation='vertical')
        plt.ylim([-10, 25])
        plt.xlim([90, 130])
        plt.title('%s Precip winter-summer mean' %run['label'])
        gl = ax.gridlines(draw_labels=True, alpha=0.5)
        gl.xlabels_top = False
        gl.ylabels_right = False
        fig_name = '%s_precip_winter_minus_summer_mean.png' %runid
        plt.savefig(fig_name)
        plt.close()
        print('Plotted %s' %fig_name)
        
        fig = plt.figure(2, figsize=(12, 7), dpi=100)
        plt.subplot(111)
        ax = plt.axes(projection=ccrs.PlateCarree())
        levels = np.linspace(-2, 2, 11)
        cmap = plt.get_cmap('RdBu')
        norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)
        cf = iplt.pcolormesh(winter_var/summer_var, cmap=cmap, norm=norm)
        plt.gca().coastlines()
        plt.colorbar(cf, orientation='vertical')
        plt.ylim([-10, 25])
        plt.xlim([90, 130])
        plt.title('%s Precip winter/summer variance' %run['label'])
        gl = ax.gridlines(draw_labels=True, alpha=0.5)
        gl.xlabels_top = False
        gl.ylabels_right = False
        fig_name = '%s_precip_winter_summer_ratio_variance.png' %runid
        plt.savefig(fig_name)
        plt.close()
        print('Plotted %s' %fig_name)

In [63]:
def plot_precip_winter_summer_bias_maps(runs, season='NDJFM', variable='mean'):
    
    for run in runs:
        runid = run['runid']
        print(run['label'])
        outfile_basename = '/project/MJO_GCSS/hadgem3/data/SEAPy_output/obs/obs_%s_precipitation_flux' %(season)
        obs_precip_cube = iris.load_cube(outfile_basename+'_%s.nc' %variable)
        out_plot_dir = os.path.join('/project/MJO_GCSS/hadgem3/data/SEAPy_output/', runid)
        if not runid=='obs':
            outfile_basename = os.path.join(out_plot_dir, '%s_%s_precipitation_flux' %(runid, season))
            precip_cube = iris.load_cube(outfile_basename+'_%s.nc' %variable)
            precip_cube.convert_units('kg m-2 day-1')
            precip_cube = precip_cube.intersection(longitude=(90, 130), latitude=(-10,25))
            precip_cube = prepare_calendar(precip_cube)

            # Correcting coordinate system
            for cc in ['latitude', 'longitude']:
                obs_precip_cube.coord(cc).coord_system = precip_cube.coord(cc).coord_system
            
            precip_cube = precip_cube.regrid(obs_precip_cube, iris.analysis.Linear())
            diff = precip_cube.copy()
            diff.data -= obs_precip_cube.data
            
            fig = plt.figure(1, figsize=(10, 7), dpi=100)
            plt.subplot(111)
            ax = plt.axes(projection=ccrs.PlateCarree())
            levels = np.arange(-20, 22, 2)
            cmap = plt.get_cmap('RdBu')
            norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)
            cf = iplt.pcolormesh(diff, cmap=cmap, norm=norm)
            plt.gca().coastlines()
            plt.colorbar(cf, orientation='vertical')
            plt.ylim([-10, 25])
            plt.xlim([90, 130])
            plt.title('%s Precip %s %s bias' %(run['label'], season, variable))
            gl = ax.gridlines(draw_labels=True, alpha=0.5)
            gl.xlabels_top = False
            gl.ylabels_right = False
            #plt.show()
            fig_name = '%s_precip_%s_%s_bias.png' %(runid, season, variable)
            plt.savefig(fig_name)
            plt.close()
            print('Plotted %s' %fig_name)

In [28]:
def plot_precip_filt_ratio_maps(runs, season='MJJAS', variable='variance'):
    for run in runs:
        runid = run['runid']
        print(run['label'])
        out_plot_dir = os.path.join('/project/MJO_GCSS/hadgem3/data/SEAPy_output/', runid)
        outfile_basename = os.path.join(out_plot_dir, '%s_%s_precipitation_flux' %(runid, season))
        precip_cube = iris.load_cube(outfile_basename+'_%s.nc' %variable)
        #if not runid=='obs':
        #    precip_cube.convert_units('kg m-2 day-1')
            
        precip_cube = precip_cube.intersection(longitude=(90, 130), latitude=(-10,25))
        precip_cube = prepare_calendar(precip_cube)

        outfile_basename = os.path.join(out_plot_dir, '%s_%s_precipitation_flux_filt' %(runid, season))
        precip_filt_cube = iris.load_cube(outfile_basename+'_%s.nc' %variable)

        # Correcting coordinate system
        for cc in ['latitude', 'longitude']:
            precip_filt_cube.coord(cc).coord_system = precip_cube.coord(cc).coord_system

        precip_filt_cube = precip_filt_cube.regrid(precip_cube, iris.analysis.Linear())
        ratio = precip_filt_cube.copy()
        ratio /= precip_cube
        
        fig = plt.figure(1, figsize=(12, 7), dpi=100)
        plt.subplot(111)
        ax = plt.axes(projection=ccrs.PlateCarree())
        levels = np.arange(0, 0.5, 0.1)
        cmap = plt.get_cmap('GnBu')
        norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)
        cf = iplt.pcolormesh(ratio, cmap=cmap, norm=norm)
        plt.gca().coastlines()
        plt.colorbar(cf, orientation='vertical')
        plt.ylim([-10, 25])
        plt.xlim([90, 130])
        plt.title('%s Precip %s filt/total %s ratio.png' %(run['label'], season, variable))
        gl = ax.gridlines(draw_labels=True, alpha=0.5)
        gl.xlabels_top = False
        gl.ylabels_right = False
        fig_name = '%s_precip_%s_%s_ratio.png' %(runid, season, variable)
        plt.savefig(fig_name)
        print('Plotted %s' %fig_name)
        plt.close()

In [20]:
outfile_basename = '/project/MJO_GCSS/hadgem3/data/SEAPy_output/obs/obs_%s_precipitation_flux' %('NDJFM')
obs_precip_cube = iris.load_cube(outfile_basename+'_%s.nc' %'mean')
outfile_basename = '/project/MJO_GCSS/hadgem3/data/SEAPy_output/u-ch221/u-ch221_%s_precipitation_flux' %('NDJFM')
precip_cube = iris.load_cube(outfile_basename+'_%s.nc' %'mean')

In [207]:
#obs_precip_cube.attributes['grid_type']
#from iris.experimental.equalise_cubes import equalise_attributes
#equalise_attributes([obs_precip_cube, precip_cube])
#obs_precip_cube = obs_precip_cube.regrid(precip_cube, iris.analysis.Linear())

In [139]:
fig = plt.figure(1, figsize=(12, 7), dpi=100)
plt.subplot(111)
ax = plt.axes(projection=ccrs.PlateCarree())
levels = np.arange(-20, 20, 2)
cmap = plt.get_cmap('RdBu')
norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)
cf = iplt.pcolormesh(winter_mean-summer_mean, cmap=cmap, norm=norm)
plt.gca().coastlines()
plt.colorbar(cf, orientation='vertical')
plt.ylim([-10, 25])
plt.xlim([90, 130])
plt.title('%s Precip winter-summer mean' %run['label'])
gl = ax.gridlines(draw_labels=True, alpha=0.5)
gl.xlabels_top = False
gl.ylabels_right = False
plt.show()

/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1193: UserWarning: Coordinate 'longitude' is not bounded, guessing contiguous bounds.
  'contiguous bounds.'.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1193: UserWarning: Coordinate 'latitude' is not bounded, guessing contiguous bounds.
  'contiguous bounds.'.format(self.name()))


In [137]:
fig = plt.figure(2, figsize=(12, 7), dpi=100)
plt.subplot(111)
ax = plt.axes(projection=ccrs.PlateCarree())
levels = np.linspace(-2, 2, 11)
cmap = plt.get_cmap('RdBu')
norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)
cf = iplt.pcolormesh(winter_var/summer_var, cmap=cmap, norm=norm)
plt.gca().coastlines()
plt.colorbar(cf, orientation='vertical')
plt.ylim([-10, 25])
plt.xlim([90, 130])
plt.title('%s Precip winter/summer variance' %run['label'])
gl = ax.gridlines(draw_labels=True, alpha=0.5)
gl.xlabels_top = False
gl.ylabels_right = False
plt.show()

/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1193: UserWarning: Coordinate 'longitude' is not bounded, guessing contiguous bounds.
  'contiguous bounds.'.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1193: UserWarning: Coordinate 'latitude' is not bounded, guessing contiguous bounds.
  'contiguous bounds.'.format(self.name()))


In [71]:
control = {}
control['runid'] = 'u-bs902'
control['label'] = 'GC4.0 N216O25'
control['start_date'] = '1981/12/01'
control['end_date'] = '2008/12/01'
control['data_retrieve_dir'] = '/scratch/hadpx/hadgem3/data/SEAPy'

expt = {}
expt['runid'] = 'u-co779'
expt['label'] = 'GC5.0 N216O25'
expt['start_date'] = '1981/12/01'
expt['end_date'] = '2008/12/01'
expt['data_retrieve_dir'] = '/scratch/hadpx/hadgem3/data/SEAPy'

obs = {}
obs['runid'] = 'obs'
obs['label'] = 'ERA5/GPM'
obs['start_date'] = '1989/01/01'
obs['end_date'] = '2008/12/01'
obs['data_retrieve_dir'] = '/project/MJO_GCSS/hadgem3/data/obs/SEAPy_data'

In [72]:
runs = [control, expt, obs]
plot_precip_monthly_means(runs)

In [70]:
plot_precip_winter_summer_maps(runs, filt=False)

GC4.0 N216O25
Mean Written /project/MJO_GCSS/hadgem3/data/SEAPy_output/u-bs902/u-bs902_DJF_precipitation_flux_mean.nc
Variance Written /project/MJO_GCSS/hadgem3/data/SEAPy_output/u-bs902/u-bs902_DJF_precipitation_flux_variance.nc
Mean Written /project/MJO_GCSS/hadgem3/data/SEAPy_output/u-bs902/u-bs902_JJA_precipitation_flux_mean.nc
Variance Written /project/MJO_GCSS/hadgem3/data/SEAPy_output/u-bs902/u-bs902_JJA_precipitation_flux_variance.nc
Plotted u-bs902_precip_winter_minus_summer_mean.png
Plotted u-bs902_precip_winter_summer_ratio_variance.png
GC5.0 N216O25
Mean Written /project/MJO_GCSS/hadgem3/data/SEAPy_output/u-co779/u-co779_DJF_precipitation_flux_mean.nc
Variance Written /project/MJO_GCSS/hadgem3/data/SEAPy_output/u-co779/u-co779_DJF_precipitation_flux_variance.nc
Mean Written /project/MJO_GCSS/hadgem3/data/SEAPy_output/u-co779/u-co779_JJA_precipitation_flux_mean.nc
Variance Written /project/MJO_GCSS/hadgem3/data/SEAPy_output/u-co779/u-co779_JJA_precipitation_flux_variance.nc


In [38]:
plot_precip_winter_summer_bias_maps(runs, season='NDJFM', variable='mean')
plot_precip_winter_summer_bias_maps(runs, season='MJJAS', variable='mean')
plot_precip_winter_summer_bias_maps(runs, season='DJF', variable='mean')
plot_precip_winter_summer_bias_maps(runs, season='JJA', variable='mean')

GC4.0 N216O25
Plotted u-bs902_precip_NDJFM_mean_bias.png
GC5.0 N216O25
Plotted u-co779_precip_NDJFM_mean_bias.png
ERAInt/TRMM
GC4.0 N216O25
Plotted u-bs902_precip_MJJAS_mean_bias.png
GC5.0 N216O25
Plotted u-co779_precip_MJJAS_mean_bias.png
ERAInt/TRMM
GC4.0 N216O25
Plotted u-bs902_precip_DJF_mean_bias.png
GC5.0 N216O25
Plotted u-co779_precip_DJF_mean_bias.png
ERAInt/TRMM
GC4.0 N216O25
Plotted u-bs902_precip_JJA_mean_bias.png
GC5.0 N216O25
Plotted u-co779_precip_JJA_mean_bias.png
ERAInt/TRMM


In [267]:
plot_precip_filt_ratio_maps(runs, season='MJJAS', variable='variance')
plot_precip_filt_ratio_maps(runs, season='NDJFM', variable='variance')

GA8GL9


/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1193: UserWarning: Coordinate 'longitude' is not bounded, guessing contiguous bounds.
  'contiguous bounds.'.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1193: UserWarning: Coordinate 'latitude' is not bounded, guessing contiguous bounds.
  'contiguous bounds.'.format(self.name()))


Plotted u-bu357_precip_MJJAS_variance_ratio.png
GA8GL9_577.6


/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1193: UserWarning: Coordinate 'longitude' is not bounded, guessing contiguous bounds.
  'contiguous bounds.'.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1193: UserWarning: Coordinate 'latitude' is not bounded, guessing contiguous bounds.
  'contiguous bounds.'.format(self.name()))


Plotted u-ci336_precip_MJJAS_variance_ratio.png
CoMv8p1


...

Plotted u-ci336_precip_NDJFM_variance_ratio.png
CoMv8p1


/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1193: UserWarning: Coordinate 'longitude' is not bounded, guessing contiguous bounds.
  'contiguous bounds.'.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1193: UserWarning: Coordinate 'latitude' is not bounded, guessing contiguous bounds.
  'contiguous bounds.'.format(self.name()))


Plotted u-ch221_precip_NDJFM_variance_ratio.png
TRMM


/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1193: UserWarning: Coordinate 'longitude' is not bounded, guessing contiguous bounds.
  'contiguous bounds.'.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1193: UserWarning: Coordinate 'latitude' is not bounded, guessing contiguous bounds.
  'contiguous bounds.'.format(self.name()))


Plotted obs_precip_NDJFM_variance_ratio.png


In [58]:
monthly_clims[-1].coord('month_number')

AuxCoord(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]), standard_name=None, units=Unit('1'), long_name='month_number')

In [64]:
np.arange(13)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])